# NLP. Lab 6. Word sense disambiguation (WSD). Lesk algorithm

[Competition link](https://www.kaggle.com/t/c2b819e33879426c94a91b7e77f35af1)

## 1. Wordnet structure in nltk

`nltk` [WordNet](https://www.nltk.org/howto/wordnet.html) is a convenient tool for working with WordNet corpus - one of the largest lexical database on English:

> Nouns, verbs, adjectives and adverbs are grouped into sets of cognitive synonyms (synsets), each expressing a distinct concept. Synsets are interlinked by means of conceptual-semantic and lexical relations. *[link](https://wordnet.princeton.edu)*

That means, the one can get the sense(s) of a word from the WordNet, and considering the context, disambiguate the most appropriate meaning of the word. Also synsets are useful for comparing words, for example, finding the most similar ones. In comparison to Word2Vec, WordNet distinguesh between antonyms and synonyms (and also alsohypernyms and hyponyms) because of manual labeling of words by the linguists.

In [1]:
import pandas as pd
import numpy as np
import nltk
import subprocess
nltk.download('all')

try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /usr/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /usr/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron

## 3. Competition. Implementation of Lesk algorithm(s) from scratch

We've seen that Lesk algorithm sometimes works incorrectly, at least, its `nltk` implementation. However, it can be modified in the ways of (1) calculation of overlapping; (2) considering PoS of a ambigous word; (3) lemmatization of words, ...

Your task (both for class and competition) on this week is to finish the implementation of Lesk algorithm

In [2]:
from nltk import pos_tag
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import sent_tokenize
import re, math
from collections import Counter
from itertools import chain
import pandas as pd

Lesk algorithm compares the overlapping of the sentence with the **signature** of every lemma of ambigous word (optionally, all lemmas with given PoS tag). By the signature we mean set of words from definitions of ambigous word. The one can extend it synset examples in WordNet; hyper-/hyponyms and  signatures from the related senses. 

We provide you pre-computed signatures in *signatures.pkl*. You can access to a signature by a synset name and pocking one of the signature "modes", i.e. the way they were constructed. Check the example to see the difference

In [3]:
cached_signatures = pd.read_pickle('/kaggle/input/nlp-lab-6-word-sense-disambiguation-wsd/signatures.pkl')

In [4]:
# Example of access to cached signatures
print(f"Original {cached_signatures['mouse.n.04']['original']}") # set of words from 'mouse.n.04' definition
print(f"Simple {cached_signatures['mouse.n.04']['simple']}") # set of words from 'mouse.n.04' (definition +  hypernymy and hyponymy)
print(f"Adapted {cached_signatures['mouse.n.04']['adapted']}") # set of words from 'mouse.n.04' (definition +  hypernymy and hyponymy + holonyms + meronyms +  troponym)

Original {'as', 'device', 'surface', 'a', 'that', 'ball', 'computer', ';', 'coordinates', 'move', 'your', 'the', 'bottom', 'you', 'around', 'hand-operated', 'on', 'it', 'controls', 'rolls', 'is', 'of', 'pad', 'screen', 'cursor', 'electronic'}
Simple {'device', 'surface', 'room', 'roll', 'trackball', 'coordinate', 'control', 'take', 'ball', 'computer', 'mouse', 'computer_mouse', 'move', 'bottom', 'around', 'hand-operated', 'electronic_device', 'much', 'pad', 'screen', 'cursor', 'electronic'}
Adapted {'device', 'surface', 'room', 'roll', 'trackball', 'coordinate', 'control', 'ball', 'take', 'computer', 'mouse', 'computer_mouse', 'move', 'bottom', 'around', 'hand-operated', 'mouse_button', 'electronic_device', 'much', 'pad', 'screen', 'cursor', 'electronic'}


In [5]:
# Support function
def signatures(ambiguous_word: str, pos: str = None, mode="simple") -> dict:
    """
    Takes an ambiguous word and optionally its Part-Of-Speech and returns
    a dictionary where keys are the synsets and values are sets of signatures.

    :param ambiguous_word: String, a single word.
    :param pos: String, one of 'a', 'r', 's', 'n', 'v', or None.
    :return: dict(synset:{signatures}).
    """
    if mode not in ["original", "simple", "adapted"]:
        raise AttributeError
    # Ensure that the POS is supported.
    pos = pos if pos in ['a', 'r', 's', 'n', 'v', None] else None

    # If the POS specified isn't found but other POS is in wordnet.
    if not wn.synsets(ambiguous_word, pos) and wn.synsets(ambiguous_word):
        pos = None

    # Holds the synset->signature dictionary
    ss_sign = {}
    for sysnset in wn.synsets(ambiguous_word, pos):
        ss_sign[sysnset] = cached_signatures[sysnset.name()][mode]

    # Matching exact words may cause sparsity, so do optional stemming
    return {ss:[PorterStemmer().stem(s) for s in signature] for ss, signature in ss_sign.items()}

In [6]:
print(signatures("button", 'n', mode='simple'))

{Synset('button.n.01'): ['shirt_button', 'fix', 'sewn', 'fasten', 'round', 'fasten', 'coat_button', 'shirt', 'fit', 'holdfast', 'buttonhol', 'coat', 'etc', 'button'], Synset('push_button.n.01'): ['bed', 'horn_button', 'doorbel', 'push_button', 'besid', 'electr', 'elev', 'press', 'bell', 'reset_button', 'electrical_switch', 'desk', 'button', 'oper', 'switch', 'buzzer', 'mouse_button', 'bell_push', 'push', 'panic_button', 'electric_switch'], Synset('button.n.03'): ['resembl', 'plant_part', 'variou', 'part', 'plant_structur', 'button', 'plant'], Synset('button.n.04'): ['pass', 'flat', 'round', 'onto', 'pin', 'garment', 'inform', 'candid', 'suitabl', 'badg', 'display', 'button', 'campaign'], Synset('clitoris.n.01'): ['peni', 'organ', 'femal', 'erectile_organ', 'clit', 'sexual', 'clitori', 'homolog', 'button'], Synset('release.n.08'): ['press', 'devic', 'mechan', 'part', 'releas', 'button'], Synset('button.n.07'): ['resembl', 'artefact', 'button', 'artifact']}


### 3.1 [Vanila Lesk]()

Algorithm:

0) *(optional) Get lemma form of the ambiguous word and use it instead of word;*
1) Get the signatures of the ambiguous words synset, i.e. the definition of *every synset* for the word;
2) *(optional) On the previous step, consider PoS tagging;*
3) *(optional) Apply lemmatization on the context as well for better robustness;*
4) For every pair of synset and its signature, calculate intersection between the context and the signature;
5) Save a synset with the biggest intersection.

In [7]:
def tmp(context):
    stemmed_context = [PorterStemmer().stem(s.lower()) for s in context.split()]
    print(stemmed_context)

tmp("I went to the bank to deposit my money")

['i', 'went', 'to', 'the', 'bank', 'to', 'deposit', 'my', 'money']


In [8]:
def split_stem(context):
    return [PorterStemmer().stem(s) for s in context.split()]

def overlap(signature, context):
    stemmed_context = set(split_stem(context))
    overlap_count = sum(1 for word in signature if word in stemmed_context)
    return overlap_count

def original_lesk(context_sentence: str, ambiguous_word: str) -> wn.synset:
    """
    Implementation of the original Lesk algorithm (1986).

    :param context_sentence: String, sentence or document.
    :param ambiguous_word: String, a single word.
    :return: A Synset for the estimated best sense.
    """
    candidates = signatures(ambiguous_word)

    max_overlap = 0
    best_synset = None
    for synset, signature in candidates.items():
        overlapped = overlap(signature, context_sentence)
        if overlapped > max_overlap:
            max_overlap = overlapped
            best_synset = synset
    
    return best_synset


In [9]:
vanila_lesk_definition = original_lesk('I went to the bank to deposit my money', 'bank').definition()
assert vanila_lesk_definition == 'a financial institution that accepts deposits and channels the money into lending activities'

### 3.2 Cosine Lesk

The initial version of the Lesk algorithm can be slightly updated in the overlap calculation part. Leave steps 0-3 as in the section.  As overlapping, calculate a cosine similarity between a signature and context. Finally, return a synset with the highest similarity of its signature with the given context. 

How to define a cos sim for the sentences?

$$cos(A,B) = \frac{\sum(A_i, B_i)}{\sum(A_i^2) * \sum(B_i^2)}$$

For example, you can count every word in context and signature. Multiply the counts of the intersected words between context and signature. Sum these multiplications, that would represent $\sum(A_i, B_i)$ in the formula above. Normalize it by counts of every words in the sentences.

In [10]:
from collections import Counter
import math

def cosine_similarity(sent1: str, sent2: str) -> float:
    """
    Calculates cosine similarity between 2 sentences
    """
    freq1 = Counter(split_stem(sent1))
    freq2 = Counter(split_stem(sent2))

    intersect = set(freq1.keys()).union(set(freq2.keys()))

    vec1 = [freq1[word] for word in intersect]
    vec2 = [freq2[word] for word in intersect]

    dot = sum([v1 * v2 for v1, v2 in zip(vec1, vec2)])
    
    magnitude1 = math.sqrt(sum([v ** 2 for v in vec1]))
    magnitude2 = math.sqrt(sum([v ** 2 for v in vec2]))

    if magnitude1 == 0 or magnitude2 == 0:
        return .0

    cosine = dot / (magnitude1 * magnitude2)
    
    # Write your code here
    return cosine

In [11]:
# mouse: 3, house: 1, claus: 2
sent1 = "mouse mouse house mouse claus claus"
# mouse: 2, house: 1, faust: 2 
sent2 = "faust house faust mouse mouse"

# cos_sim = (3*2 + 1*1) / [sqrt((3^2 + 1^2 2^2 ))*sqrt((2^2 + 1^2 + 2^2))] = 7 / 11.22 ~= 0.62
print(cosine_similarity(sent1, sent2))

0.6236095644623235


In [12]:
def cosine_lesk(context_sentence: str, ambiguous_word: str, pos: str = None) -> wn.synset:
    """
    In line with vector space models, we can use cosine to calculate overlaps
    instead of using raw overlap counts. Essentially, the idea of using
    signatures (aka 'sense paraphrases') is lesk-like.

    :param context_sentence: String, sentence or document.
    :param ambiguous_word: String, a single word.
    :param pos: String, one of 'a', 'r', 's', 'n', 'v', or None.
    :return: A Synset for the estimated best sense.
    """

    candidates = signatures(ambiguous_word)

    max_overlap = 0.
    best_synset = None
    for synset, signature in candidates.items():
        overlapped = cosine_similarity(" ".join(signature), context_sentence)
        if overlapped > max_overlap:
            max_overlap = overlapped
            best_synset = synset
    
    return best_synset

In [13]:
cosine_lesk_definition = cosine_lesk('I went to the bank to deposit my money', 'bank').definition()
assert (cosine_lesk_definition == 'a building in which the business of banking transacted')

### 3.3 Adapted Lesk

Another improvement of original Lesk algorithm is considering *wider* signature. As you know, we considered the words from definition as signature before. What if we include words from the neighbour relations of the synsets? That could be their  hypernymy and hyponymy, holonyms, meronyms and  troponym. What is the main idea of **adapted** Lesk algorithm. 

We don't ask you to collect all these relations from scratch - precomputed signatures contains in `signatures.pkl["adapted"]`. Repeat the implementation of original Lesk with these signatures and compare their performance

In [14]:
from nltk.tokenize import TreebankWordTokenizer
from nltk import pos_tag


def adapted_lesk(context_sentence: str, ambiguous_word: str,
                pos: str = None) -> wn.synset:
    """
    This function is the implementation of the Adapted Lesk algorithm,
    described in Banerjee and Pederson (2002). It makes use of the lexical
    items from semantically related senses within the wordnet
    hierarchies and to generate more lexical items for each sense.
    see www.d.umn.edu/~tpederse/Pubs/cicling2002-b.pdf

    You don't need to apply window on every word to get its signature, as 

    
    :param context_sentence: String, sentence or document.
    :param ambiguous_word: String, a single word.
    :param pos: String, one of 'a', 'r', 's', 'n', 'v', or None.
    :return: A synset for the estimated best sense.
    """

    candidates = signatures(ambiguous_word, mode= "adapted")

    max_overlap = 0
    best_synset = list(candidates.keys())[0]
    for synset, signature in candidates.items():
        overlapped = overlap(signature, context_sentence)
        if overlapped > max_overlap:
            max_overlap = overlapped
            best_synset = synset
    
    return best_synset


In [15]:
adapted_lesk_definition = adapted_lesk('I went to the bank to deposit my money', 'bank').definition()
assert (adapted_lesk_definition == 'a financial institution that accepts deposits and channels the money into lending activities')

### 3.4 Competition

As a competition, you're asked to return a definition from WordNet dictionary for an ambigous word with respect to its context. You're given only raw sentences (with punctuation and other junk) and the target word. `val.csv` contains some examples on which you can test the performance of your solution.

**Note:** You can use only modifications of Lesk algorithm on this competition, i.e. BERT-based WSD solvers are allowed. However, you can use PoS taggers from other libraries to improve the performance of the implementation.

In [16]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.8 MB/s eta 0:00:00


In [17]:
import pandas as pd
from Levenshtein import distance

df_val = pd.read_csv('/kaggle/input/nlp-lab-6-word-sense-disambiguation-wsd/val.csv')

df_val.head()

,sentence,ambigous word,definition
0,joan beck is right on target when she says tha...,hard,dispassionate
1,sugar refiners attracted buying interest after...,interest,a sense of concern with and curiosity about so...
2,"signs of a weakening economy, such as friday '...",interest,a fixed charge for borrowing money; usually a ...
3,stir in the bread crumbs and season to taste w...,serve,provide (usually but not necessarily food)
4,"tom fast, of scotts valley, stepped out of the...",hard,dispassionate


In [18]:
total_distance = 0.

for _, row in df_val.iterrows():
    pred = adapted_lesk(row['sentence'], row['ambigous word']).definition()
    gold = row['definition']
    total_distance += distance(pred, gold)

print("Average Levenshtein distance:", total_distance / len(df_val))

Average Levenshtein distance: 41.84444444444444


In [19]:
df_test = pd.read_csv("/kaggle/input/nlp-lab-6-word-sense-disambiguation-wsd/test.csv")
pred = []
for _, row in df_test.iterrows():
    prediction = adapted_lesk(row['sentence'], row["ambigous word"]).definition()
    pred.append(prediction)

In [20]:
df_test["prediction"] = pred
df_test = df_test.drop(["ambigous word", "sentence"],axis=1)
df_test.to_csv("submission.csv",index_label="ID", index=False)